In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import math
from statistics import mean
from matplotlib import cm

import warnings
warnings.simplefilter("ignore")

## Figure 5

### Max cluster size

In [ ]:
df= pd.read_json('df_sims_10-21_index.JSON')

In [ ]:
plt.rcParams['figure.figsize'] = 8,8
ds_gg = df.groupby('density')
for ds, ds_df in ds_gg:
    df_heatmap = ds_df.pivot_table(values='max_size',
                            index='psi',
                            columns='Wc', aggfunc=np.mean)
    df_heatmap = df_heatmap.sort_index(axis=0, ascending=False)

    fig, ax = plt.subplots()

    im = ax.imshow(df_heatmap,vmin=1, vmax=500)#vmax=df_heatmap.max().max())

    # We want to show all ticks...
    ax.set_xticks(np.arange(len(df_heatmap.columns)))
    ax.set_yticks(np.arange(len(df_heatmap.index)))
    # ... and label them with the respective list entries
    ax.set_xticklabels(df_heatmap.columns, fontsize=16)
    ax.set_yticklabels(df_heatmap.index, fontsize=18)
    # Set axis titles
    ax.set_xlabel("$W_c$", fontsize=26)
    #ax.set_ylabel("$\psi$", fontsize=26)
    
    # colorbar set up
    cb = plt.colorbar(im,fraction=0.046, pad=0.04)
    cb.ax.tick_params(labelsize=18)
    cb.set_label("Maximum cluster size",size=20)

    ax.set_title(f"$\phi$={round(ds, 2)}", fontsize=26)
    fig.tight_layout()
    fig.savefig(f"heatmap__dens__{round(ds, 2)}.png", dpi=300)
plt.show()

## Figure 6

In [ ]:
df = pd.read_json('df_sims_09_24.JSON')

### Slopes for different CILs

In [ ]:
plt.rcParams['figure.figsize'] = 5, 4
df_D01 = df[(df.D==1.0)&(df.Wc==1)].groupby(['psi','density'
            ]).size().reset_index().rename(columns={0:'Count'})

for id, col in df_D01.iterrows():
    legend = []
    df_plot = df[(df.D==1.0)&(df.Wc==1)&(df.density==col['density'])]
    for psi, df_psi in df_plot.groupby('psi'):
        plotdata = df_psi.groupby('time').mean()
        plt.loglog(plotdata.index, plotdata.max_size, '.', label=f"$\psi$={psi}")
        
        A, B = np.polyfit(np.log10(plotdata.index[1:]), np.log10(plotdata.max_size[1:]), 1)
        print(A)
        plt.loglog(plotdata.index[1:], 10**(np.log10(plotdata.index)[1:]*A+B),'--',color='k',label='_nolegend_')
        legend.append(psi)
        
   
    plt.title(f"$\phi$={round(col['density'],2)}")
    plt.xlabel('Time',fontsize=13)
    plt.ylabel('N° of Clusters',fontsize=13)
    
    plt.legend(legend, title=r'$\psi$')
    #plt.savefig(f"maxsize_log__dens__{round(col['density'],2)}.png")
    plt.show()

## Figure 8

### Dynamics

In [ ]:
df = pd.read_json('df_chem_05_31.JSON')

In [ ]:
plt.rcParams['figure.figsize'] = 7, 5
# params dataframe
df_D01 = df[(df.D==1)&(df.psi.isin([0,2]))&(df.forg.isin([0.01, 0.1, 1]))].groupby(['Wc', 'forg']).size().reset_index().rename(columns={0:'Count'})

for id, col in df_D01.iterrows():
    legend = []
    df_plot = df[(df.Wc==col['Wc'])&(df.forg==col['forg'])]
    for Wc, df_Wc in df_plot.groupby('Wc'):
        for forg, df_forg in df_Wc.groupby('forg'):
            for psi, df_psi in df_forg.groupby('psi'):
                print(f"Params: Wc={Wc}, forg={forg}, psi={psi}")
                plotdata = df_psi.groupby('time').mean()
                plt.loglog(plotdata.index, plotdata.max_size, '.', label=f"$\psi$={psi}", markeredgewidth=4)
                """
                # curve fitting
                A, B = np.polyfit(np.log10(plotdata.index[1:]), np.log10(plotdata.max_size[1:]), 1)
                print(f"Fit params: A={A}, B={B}")
                plt.loglog(plotdata.index[1:], 10**(np.log10(plotdata.index)[1:]*A+B),'--',color='k',label='_nolegend_', linewidth=4)
                """
                legend.append(round(psi))
                
   
            #plt.title(f"$\phi$={round(col['density'],2)}", fontsize=20)
            plt.ylim(1,500)
            plt.xlabel('Time',fontsize=20)
            plt.ylabel('Maximum cluster size',fontsize=20)
            plt.xticks(fontsize=14)
            plt.yticks(fontsize=14)
            plt.tight_layout()

            plt.legend(legend, title=r'$\psi$')
            plt.savefig(f"maxsize_Wc__{Wc}__forg__{forg}.png")
            plt.show()